In [1]:
from sklearn.feature_extraction.text import TfidfVectorizer
import os
import glob as glob
import pandas as pd
import re
from nltk.stem import WordNetLemmatizer
from sklearn.metrics.pairwise import cosine_similarity

##### Read all documents which are added to text.tsv
This text.tcv is created by textvectoring file

In [2]:
text = pd.read_csv('data/text.tsv', delimiter='\t', header=None, names=["sequence", "text"])
text.head()

,sequence,text
0,1,sandoz plans weedkiller joint venture in ussr....
1,2,taiwan rejects textile makers exchange rate pl...
2,3,national fsi inc nfsi> 4th qtr loss. shr loss ...
3,4,"occidental oxy> official resigns. midcon corp,..."
4,5,italy's bnl to issue 120 mln dlr convertible b...


##### Read respective tags of documents

In [3]:
tags = pd.read_csv('data/oldfiles/tags.tsv', delimiter='\t', header=None, names=["sequence", "tags"])
tags.head()

,sequence,tags
0,1,"usa,ussr"
1,2,"usa,taiwan"
2,3,"earn,usa"
3,4,usa
4,5,italy


##### create lemmtized text to root word

##### Add tags to text file

In [4]:
text['tags'] = tags['tags']

##### binary encode the tags

In [5]:
from sklearn.preprocessing import LabelBinarizer


lb = LabelBinarizer()
Y = lb.fit_transform(text.tags)
Y

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ..., 
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

##### Create TFidfVectore

In [6]:
# below parameters gives correct cosine prediction
vectorizer = TfidfVectorizer(
    max_df=0.5,
    max_features=5000,
    min_df=10,
    stop_words='english',
    use_idf=True)

X = vectorizer.fit_transform(text['text'].str.lower())
print(X.shape)
tfidf = X

(19716, 5000)


##### Calcuate Cosine similarity 

In [7]:
#pairwise_similarity = ((tfidf * tfidf.T).A)
pairwise_similarity = cosine_similarity(X)
print(pairwise_similarity.shape)
dist = 1 - pairwise_similarity
print(dist.shape)

(19716, 19716)
(19716, 19716)


##### Applying Dimention Reduction - PCA

In [8]:
from sklearn.decomposition import PCA
pca = PCA(n_components=2)
pos = pca.fit_transform(dist)
xs, ys = pos[:,0], pos[:,1]

In [9]:
X = pos


##### Get values which have cosine greater than 0.32 for first document

In [ ]:
#For max_feature = 5000
for j in range(20):
    for i in range(len(pairwise_similarity)):
        if( pairwise_similarity[j][i] > 0.32):
            print(i , " ", pairwise_similarity[j][i], " " , tags['tags'][i])
    print('*******************************************************************')

In [ ]:
for i in range(len(pairwise_similarity)):
        if( pairwise_similarity[116][i] > 0.32):
            print(i , " ", pairwise_similarity[116][i], " " , tags['tags'][i])

cosine silimarity more than 0.32 is predicting correct documents.

##### split into train and test set

In [29]:
N = int(.8 * X.shape[0])
Xtr, ytr = X[:N], Y[:N]
Xte, yte = X[N:], Y[N:]

 just ignore warnings of ill-defined recall/precision etc.

In [30]:
import warnings
warnings.filterwarnings("ignore")

##### Apply Logistic Regression

In [31]:
from sklearn.multiclass import OneVsRestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.grid_search import GridSearchCV
import numpy as np

# logistic regression parameter to optimise
params = {"estimator__C": np.logspace(1, 1.5, num=5)}
# use OneVsRestClassifier for multiclass learning
model = OneVsRestClassifier(LogisticRegression())

In [20]:
from sklearn.ensemble import GradientBoostingClassifier  #GBM algorithm
from sklearn import cross_validation, metrics   #Additional scklearn functions
from sklearn.grid_search import GridSearchCV   #Perforing grid search
import numpy as np

# use OneVsRestClassifier for multiclass learning
model = OneVsRestClassifier(GradientBoostingClassifier(random_state=10))

In [34]:
from sklearn.linear_model import SGDClassifier
from sklearn import cross_validation, metrics   #Additional scklearn functions
from sklearn.grid_search import GridSearchCV   #Perforing grid search
import numpy as np

# use OneVsRestClassifier for multiclass learning
model = OneVsRestClassifier(SGDClassifier())

In [35]:
fit = model.fit(Xtr,ytr)

In [ ]:
Xte.toarray()

In [36]:
ypred = fit.predict(Xte)

In [37]:
len(yte)

3944

In [38]:
tags = []
for n in xrange(len(yte)):
    tags.append((lb.classes_[yte[n]==1], lb.classes_[ypred[n]==1]))
    
ans = pd.DataFrame(tags, columns=['actualtags', 'predictedtags'])

In [39]:
ans.head(300)

,actualtags,predictedtags
0,"[earn,italy]",[]
1,[uk],[]
2,"[nickel,uk]",[]
3,[hong-kong],[usa]
4,"[earn,netherlands]",[]
5,"[money-fx,dlr,usa,west-germany,poehl]",[usa]
6,"[acq,usa]",[]
7,"[earn,usa]",[]
8,"[earn,usa]","[earn,usa]"
9,[japan],"[interest,usa, usa]"


##### do a grid search on the params, with 5-fold cross-validation

In [40]:
from sklearn.metrics import accuracy_score
accuracy_score(yte, ypred)

0.14934077079107505

##### Compute F1 Score usinf TP,FP,FN

In [ ]:
from sklearn.metrics import f1_score

# compute predictions on test set
pred = clf.predict(Xte)
# compute F1-score on test set
f1_score(yte, pred)

In [ ]:
# compare with tags in test data
tags = []
for n in xrange(20):
    tags.append((lb.classes_[yte[n]==1], lb.classes_[pred[n]==1]))
    
pd.DataFrame(tags, columns=['actual tags', 'predicted tags'])

## Another way to get cosine similarity